# Slicing

Users coming from Python, and in particular NumPy, are familiar and used to powerful slicing operations. I tried to mirror that behavior for `RandomAccessibleInterval` to some extent and adopt in ntakt as much as possible. As of now, only basic slicing is supported. Unfortunately, the very concise Python syntax for slicing, i.e.
```python
start:stop:step
```
where any of `start`, `stop`, `step` can be omitted, cannot be translated to Kotlin verbatim. Instead, I introduced a `Slicing` interface that can be either an `Ellipsis` (equivalent to `...` in Python) or a `Slice` (equivalent to `slice` in Python). For brevity, I introduced the following convenience functions with Python equivalent in comments (all functions are overloaded for combinations of `Long`/`Int` but only the `Long` overloads are presented, for brevity):
```Kotlin
infix fun Long.sl(stop: Long): Slice   // this:stop
infix fun Slice.st(step: Long): Slice  // slice.start:slice.stop:step
val Long.start: Slice                  // this:
val Long.stop: Slice                   // :this
val Long.step: Slice                   // ::this
val Long.pos: Position                 // this (pin dim to index by scalar; reduces number of dims by 1)
```
The `Ellipsis` has a single instance that can be accessed via `_el` (name may change in the future).

**Note**: `min` and `max` of a `RandomAccessibleInterval` are not considered and `RandomAccessibleInterval.isZeroMin` is enforced.

In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("org.ntakt:ntakt:0.1.5-SNAPSHOT")

// uncomment to search in jitpack
// @file:DependsOn("org.ntakt:ntakt:main-SNAPSHOT")

In [2]:
import org.ntakt.*

## Examples

### 1D

In [3]:
val img = ntakt.ints(10) { it }
println(img.flatStringRepresentation)
println(img[_el].flatStringRepresentation)
println(img[slice()].flatStringRepresentation)
println(img[3.start].flatStringRepresentation)
println(img.translate(1345315L)[3.start].flatStringRepresentation)
println(img[7.stop].flatStringRepresentation)
println(img[3.step].flatStringRepresentation)
println(img[(-3).step].flatStringRepresentation)
println(img[1 sl 9].flatStringRepresentation)
println(img[1 sl -1].flatStringRepresentation)
println(img[1 sl 9 st 5].flatStringRepresentation)
println(img[1 sl -1 st 5].flatStringRepresentation)
println(img[1 sl 9 st -5].flatStringRepresentation)
println(img[1 sl -1 st -5].flatStringRepresentation)
println(img[1 sl 1].flatStringRepresentation)

ArrayImg [10]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
net.imglib2.view.SubsampleIntervalView@537d77ba: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
net.imglib2.view.SubsampleIntervalView@376427b5: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
net.imglib2.view.SubsampleIntervalView@5bf9b4fb: [3, 4, 5, 6, 7, 8, 9]
net.imglib2.view.SubsampleIntervalView@620454d8: [3, 4, 5, 6, 7, 8, 9]
net.imglib2.view.SubsampleIntervalView@1f873b51: [0, 1, 2, 3, 4, 5, 6]
net.imglib2.view.SubsampleIntervalView@195d5bb6: [0, 3, 6, 9]
net.imglib2.view.SubsampleIntervalView@1929b5f4: [9, 6, 3, 0]
net.imglib2.view.SubsampleIntervalView@5d5bbf63: [1, 2, 3, 4, 5, 6, 7, 8]
net.imglib2.view.SubsampleIntervalView@35e5cae8: [1, 2, 3, 4, 5, 6, 7, 8]
net.imglib2.view.SubsampleIntervalView@613c6263: [1, 6]
net.imglib2.view.SubsampleIntervalView@619c6980: [1, 6]
net.imglib2.view.SubsampleIntervalView@2fb521cb: [8, 3]
net.imglib2.view.SubsampleIntervalView@349c1c5f: [8, 3]
net.imglib2.view.SubsampleIntervalView@36315fd5: []


### 2D

In [4]:
val img = ntakt.ints(2, 3) { it }
img.hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
img[(-1).step].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
img[_el, (-1).step].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
img[(-1).step, (-1).step].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
img[(-1).step, 1 sl 3 st -1].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
img[(-1).step, 3.stop st -2].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }

IntervalView [(0) -- (1) = 2]: [0, 1]
IntervalView [(0) -- (1) = 2]: [2, 3]
IntervalView [(0) -- (1) = 2]: [4, 5]

IntervalView [(0) -- (1) = 2]: [1, 0]
IntervalView [(0) -- (1) = 2]: [3, 2]
IntervalView [(0) -- (1) = 2]: [5, 4]

IntervalView [(0) -- (1) = 2]: [4, 5]
IntervalView [(0) -- (1) = 2]: [2, 3]
IntervalView [(0) -- (1) = 2]: [0, 1]

IntervalView [(0) -- (1) = 2]: [5, 4]
IntervalView [(0) -- (1) = 2]: [3, 2]
IntervalView [(0) -- (1) = 2]: [1, 0]

IntervalView [(0) -- (1) = 2]: [5, 4]
IntervalView [(0) -- (1) = 2]: [3, 2]

IntervalView [(0) -- (1) = 2]: [5, 4]
IntervalView [(0) -- (1) = 2]: [1, 0]


### 3D with Scalar Slicing

In [5]:
val img = ntakt.ints(2, 3, 4) { it }
println("Entire data")
for (i in 0 until 4)
    img[_el, i.pos].hyperSlicesList(1).forEach { println("${it.flatStringRepresentation} (z=$i)") }
println()
println("x pinned to 1")
img[1.pos].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
println("y pinned to 0")
img[slice(), 0.pos].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
println("z pinned to 3, every other y")
img[slice(), 2.step, 3.pos].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }
println()
println("min/max are disregarded. Translate by [1, -2, 3], then z pinned to 3, every other y")
img.translate(1L, -2L, 3L)[slice(), 2.step, 3.pos].hyperSlicesList(1).forEach { println(it.flatStringRepresentation) }

Entire data
IntervalView [(0) -- (1) = 2]: [0, 1] (z=0)
IntervalView [(0) -- (1) = 2]: [2, 3] (z=0)
IntervalView [(0) -- (1) = 2]: [4, 5] (z=0)
IntervalView [(0) -- (1) = 2]: [6, 7] (z=1)
IntervalView [(0) -- (1) = 2]: [8, 9] (z=1)
IntervalView [(0) -- (1) = 2]: [10, 11] (z=1)
IntervalView [(0) -- (1) = 2]: [12, 13] (z=2)
IntervalView [(0) -- (1) = 2]: [14, 15] (z=2)
IntervalView [(0) -- (1) = 2]: [16, 17] (z=2)
IntervalView [(0) -- (1) = 2]: [18, 19] (z=3)
IntervalView [(0) -- (1) = 2]: [20, 21] (z=3)
IntervalView [(0) -- (1) = 2]: [22, 23] (z=3)

x pinned to 1
IntervalView [(0) -- (2) = 3]: [1, 3, 5]
IntervalView [(0) -- (2) = 3]: [7, 9, 11]
IntervalView [(0) -- (2) = 3]: [13, 15, 17]
IntervalView [(0) -- (2) = 3]: [19, 21, 23]

y pinned to 0
IntervalView [(0) -- (1) = 2]: [0, 1]
IntervalView [(0) -- (1) = 2]: [6, 7]
IntervalView [(0) -- (1) = 2]: [12, 13]
IntervalView [(0) -- (1) = 2]: [18, 19]

z pinned to 3, every other y
IntervalView [(0) -- (1) = 2]: [18, 19]
IntervalView [(0) -